## 필요 library import

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## dataset load

In [ ]:
# Data load
tmp1 = pd.read_csv("/content/drive/MyDrive/기상에따른_전력수요예측/electric_train.csv",
                   encoding = "latin1")
tmp2 = pd.read_csv("/content/drive/MyDrive/기상에따른_전력수요예측/electric_test.csv",
                   encoding = "latin1")


In [ ]:
tmp1.shape

(1048575, 17)

In [ ]:
# copy
df_train = tmp1.copy()
df_test = tmp2.copy()

## 전처리 전 dataset 수정

In [ ]:
# train data filter

df_train.drop(["e_n", "e_sum_qctr", "e_sum_load", "e_n_mean_load", "Unnamed: 0",
               "e_hh24"], axis = 1, inplace = True)
df_test.drop(["Unnamed: 0", "e_hh24"], axis = 1, inplace = True)

# df_train["e_tm"] 데이터 타입을 datetime으로 변환

df_train["e_tm"] = pd.to_datetime(df_train["e_tm"])
df_test["e_tm"] = pd.to_datetime(df_test["e_tm"])

In [ ]:
print(df_train.shape)
print(df_test.shape)

(1048575, 11)
(1048575, 10)


In [ ]:
# 3년 데이터 있는 c_num 추출
# list안에 c_num별로 column을 나누어 저장
lst = df_train["c_num"].unique() # train data의 모든 'c_num'값을 lst에 넣음

c_num_lst = [] # 빈 리스트 생성
for i in lst:
    df_temp = df_train[df_train["c_num"] == i ] # 각 'c_num'별로 데이터프레임 생성 후
    c_num_lst.append(df_temp)  # 빈 리스트에 저장

len(c_num_lst) # 'c_num'의 고유값 총 48개

# shape이 (26304, 11)=3개년인 데이터프레임들만 선택하여 합치기
selected_indices = []
selected_dfs = []
for i, df in enumerate(c_num_lst):
    if df.shape == (26304, 11):
        selected_indices.append(i)
        selected_dfs.append(df)

# 선택된 데이터프레임들을 하나로 합치기
dfs = pd.concat(selected_dfs, ignore_index=True)

# 선택된 인덱스와 합쳐진 데이터프레임 출력
print("Selected indices:", selected_indices)
print(dfs)
print(len(selected_dfs))

Selected indices: [1, 2, 7, 8, 10, 11, 12, 15, 16, 18, 19, 20, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 37, 38, 39, 41, 42, 43, 44, 46]
        c_num                e_tm  c_stn  c_nph_ta  c_nph_hm  c_nph_ws_10m  \
0        5565 2020-01-01 01:00:00    184       2.3      65.8           1.7   
1        5565 2020-01-01 02:00:00    184       2.5      69.7           1.8   
2        5565 2020-01-01 03:00:00    184       2.8      69.5           2.3   
3        5565 2020-01-01 04:00:00    184       2.8      66.0           1.6   
4        5565 2020-01-01 05:00:00    184       3.0      66.1           1.6   
...       ...                 ...    ...       ...       ...           ...   
841723  11122 2022-12-31 20:00:00    904       6.7      57.4           0.7   
841724  11122 2022-12-31 21:00:00    904       5.8      57.3           0.2   
841725  11122 2022-12-31 22:00:00    904       6.4      57.6           1.3   
841726  11122 2022-12-31 23:00:00    904       6.7      58.1           1.2   

In [ ]:
# DataFrame의 각 컬럼에서 고유값을 가져와 set으로 변환한 후 교집합을 구함
c_stn_elements = list(set(dfs["c_stn"].unique()) & set(df_test["c_stn"].unique()))
# [165, 774, 710, 783, 722, 788, 184, 156]이 나오지만 184=제주도라 제외
c_stn_elements = [165, 774, 710, 783, 722, 788, 156]

In [ ]:
c_stn_elements

[165, 774, 710, 783, 722, 788, 156]

In [ ]:
real_lst = []

In [ ]:
for i in c_stn_elements:
    var = i

    filtered_data_train = dfs[dfs['c_stn'] == var]

    # 필터링된 데이터에서 'c_num' 컬럼 값 출력
    c_num_values = filtered_data_train['c_num'].values

    filtered_data_test = df_test[df_test['c_stn'] == var]

    # 필터링된 데이터에서 'c_num' 컬럼 값 출력
    c_num_values2 = filtered_data_test['c_num'].values

    real_elements = list(set(c_num_values) & set(c_num_values2))

    print(real_elements)
#real_elements에서 해당하는 c_num값 real_lst에 넣기
real_lst = [9884, 9736, 9885, 10487, 11083, 11084, 10937, 10936, 11085]

    # 26304 * 10 26만개
    # 8760 * 10 8만7천

[9884]
[9736, 9885]
[10487]
[11083, 11084]
[10937]
[10936]
[11085]


In [ ]:
real_lst

[9884, 9736, 9885, 10487, 11083, 11084, 10937, 10936, 11085]

In [ ]:
# 'c_stn'이 선택한 값들에 해당하는 행들 추출
fin_train = dfs[dfs['c_num'].isin(real_lst)]
fin_test = df_test[df_test['c_num'].isin(real_lst)]

In [ ]:
fin_train.shape

(236736, 11)

In [ ]:
fin_test.shape

(78840, 10)

## 이상치 처리

In [ ]:
# PCA를 적용할 컬럼 선택
columns_to_pca = ['c_nph_ta_chi', 'c_nph_ta']

# 데이터 표준화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(fin_train[columns_to_pca])
y_scaled = scaler.fit_transform(fin_test[columns_to_pca])

# PCA 적용
pca = PCA(n_components=1)  # 1개의 주성분으로 축소
X_pca = pca.fit_transform(X_scaled)
y_pca = pca.fit_transform(y_scaled)

# 새로운 컬럼을 데이터프레임에 추가
fin_train['pca_feature'] = X_pca
fin_test['pca_feature'] = y_pca

<ipython-input-21-32c3ed85fa57>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fin_train['pca_feature'] = X_pca
<ipython-input-21-32c3ed85fa57>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fin_test['pca_feature'] = y_pca


In [ ]:
print(fin_train.shape)
print(fin_test.shape)

(236736, 12)
(78840, 11)


In [ ]:
#기온, 체감온도의 상관관계 0.96->거의 동일한 값을 가진다로 판단됨으로 pca활용 새로운 컬럼으로 대체 후 기존 기온, 체감온도 컬럼 삭제
fin_train.drop(['c_nph_ta_chi', 'c_nph_ta'], axis=1, inplace=True)
fin_test.drop(['c_nph_ta_chi', 'c_nph_ta'], axis=1, inplace=True)

<ipython-input-22-54d61a38a25b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fin_train.drop(['c_nph_ta_chi', 'c_nph_ta'], axis=1, inplace=True)
<ipython-input-22-54d61a38a25b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fin_test.drop(['c_nph_ta_chi', 'c_nph_ta'], axis=1, inplace=True)


In [ ]:
print(fin_train.info())
print(fin_test.info())

<class 'pandas.core.frame.DataFrame'>
Index: 236736 entries, 52608 to 710207
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   c_num         236736 non-null  int64         
 1   e_tm          236736 non-null  datetime64[ns]
 2   c_stn         236736 non-null  int64         
 3   c_nph_hm      236736 non-null  float64       
 4   c_nph_ws_10m  236736 non-null  float64       
 5   c_nph_rn_60m  236736 non-null  float64       
 6   e_weekday     236736 non-null  int64         
 7   e_week_name   236736 non-null  int64         
 8   e_elec        236736 non-null  float64       
 9   pca_feature   236736 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int64(4)
memory usage: 19.9 MB
None
<class 'pandas.core.frame.DataFrame'>
Index: 78840 entries, 35040 to 797159
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----     

In [ ]:
# 풍속 이상치를 제거하는 함수
# 풍속의 이상치(음수값=-99)를 전/후 평균값으로 대체
def replace_negatives_with_mean(df, column):
    values = df[column].values
    for i in range(len(values)):
        if values[i] < 0:
            if i == 0:
                next_value = values[i + 1]
                values[i] = next_value  # 첫 번째 값이 음수일 경우 다음 값으로 대체
            elif i == len(values) - 1:
                prev_value = values[i - 1]
                values[i] = prev_value  # 마지막 값이 음수일 경우 이전 값으로 대체
            else:                       # 이전 값과 다음 값의 평균으로 대체
                prev_value = values[i - 1]
                next_value = values[i + 1]
                values[i] = (prev_value + next_value) / 2
    df[column] = values

# 음수 값을 이상치로 간주하고 대체
replace_negatives_with_mean(fin_train, 'c_nph_ws_10m')
replace_negatives_with_mean(df_test, 'c_nph_ws_10m')

<ipython-input-25-e3519ce48770>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = values


In [ ]:
# 풍속 이상값이 있는지 확인
indices = fin_train[fin_train == -99].stack().index.tolist()
indices2 = df_test[fin_test == -99].stack().index.tolist()

print(indices)
print(indices2)

[]
[]


In [ ]:
#풍속의 이상치 대체 후 describe 확인
pd.options.display.float_format = '{:.4f}'.format
print(fin_train['c_nph_ws_10m'].describe())
print(df_test['c_nph_ws_10m'].describe())

count   236736.0000
mean         2.8669
std          2.2638
min          0.0000
25%          1.2000
50%          2.3000
75%          3.8000
max         20.7000
Name: c_nph_ws_10m, dtype: float64
count   1048575.0000
mean          1.6880
std           1.7839
min         -99.0000
25%           0.5000
50%           1.2000
75%           2.3000
max          30.0000
Name: c_nph_ws_10m, dtype: float64


In [ ]:
fin_train.set_index("e_tm", inplace=True)
fin_test.set_index("e_tm", inplace=True)

In [ ]:
print(fin_train.info())
print(fin_test.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 236736 entries, 2020-01-01 01:00:00 to 2023-01-01 00:00:00
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   c_num         236736 non-null  int64  
 1   c_stn         236736 non-null  int64  
 2   c_nph_hm      236736 non-null  float64
 3   c_nph_ws_10m  236736 non-null  float64
 4   c_nph_rn_60m  236736 non-null  float64
 5   e_weekday     236736 non-null  int64  
 6   e_week_name   236736 non-null  int64  
 7   e_elec        236736 non-null  float64
 8   pca_feature   236736 non-null  float64
dtypes: float64(5), int64(4)
memory usage: 18.1 MB
None
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 78840 entries, 2023-01-01 01:00:00 to 2024-01-01 00:00:00
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   c_num         78840 non-null  int64  
 1   c_stn         78840 non-null  int64  
 

## ‘c_num’별로 데이터프레임을 나눈 후 수평으로 합치기

In [ ]:
# 수평으로 합치기 train

lst = fin_train["c_num"].unique()
# list안에 c_num별로 column을 나누어 저장

c_num_lst = []
for i in lst:
    df_temp = fin_train[fin_train["c_num"] == i ]
    c_num_lst.append(df_temp)

# 각 데이터프레임의 컬럼 이름에 접미사 추가
for i in range(len(c_num_lst)):
    c_num_lst[i] = c_num_lst[i].rename(columns=lambda x: x + "_" + str(lst[i]))

# 모든 데이터프레임을 리스트에 저장
all_dfs = c_num_lst

# 데이터프레임들을 수평으로 합치기 (기존 컬럼명을 유지)
fin_train = pd.concat(all_dfs, axis=1, join='inner')

In [ ]:
# 수평으로 합치기 test
# list안에 c_num별로 column을 나누어 저장

c_num_lst = []
for i in lst:
    df_temp = fin_test[fin_test["c_num"] == i ]
    c_num_lst.append(df_temp)

# 각 데이터프레임의 컬럼 이름에 접미사 추가
for i in range(len(c_num_lst)):
    c_num_lst[i] = c_num_lst[i].rename(columns=lambda x: x + "_" + str(lst[i]))

# 모든 데이터프레임을 리스트에 저장
all_dfs = c_num_lst

# 데이터프레임들을 수평으로 합치기 (기존 컬럼명을 유지)
fin_test = pd.concat(all_dfs, axis=1, join='inner')

In [ ]:
print(fin_train.shape)
print(fin_test.shape)

(26304, 81)
(8760, 72)


In [ ]:
import re

# train c_num로 시작하는 컬럼명 추출
cnum_drop = [col for col in fin_train.columns if re.match(r'^c_num_', col)]
cstn_drop = [col for col in fin_train.columns if re.match(r'^c_stn_', col)]

# train 해당 컬럼 삭제
fin_train = fin_train.drop(columns=cnum_drop)
fin_train = fin_train.drop(columns=cstn_drop)

# test c_num로 시작하는 컬럼명 추출
cnum_drop = [col for col in fin_test.columns if re.match(r'^c_num_', col)]
cstn_drop = [col for col in fin_test.columns if re.match(r'^c_stn_', col)]

# test 해당 컬럼 삭제
fin_test = fin_test.drop(columns=cnum_drop)
fin_test = fin_test.drop(columns=cstn_drop)

In [ ]:
print(fin_train.shape)
print(fin_test.shape)

(26304, 63)
(8760, 54)


## 스케일링

In [ ]:
# 스케일링전 스케일링 필요없거나 방식 다른의 스케일링 필요 컬럼 추출 후 드롭
# 'e_weekday_9736', 'e_week_name_9736' 추출
# 'e_weekday' -> 월(0)-일(6), one-hot encoding 필요
# 'e_weekname' -> 주중(0),주말(1)
train_weekday = fin_train[['e_weekday_9736']]
train_weekname = fin_train[['e_week_name_9736']]
test_weekday = fin_test[['e_weekday_9736']]
test_weekname = fin_test[['e_week_name_9736']]

In [ ]:
# train c_num로 시작하는 컬럼명 추출
e_weekday_drop = [col for col in fin_train.columns
                  if re.match(r'^e_weekday_', col)]
e_week_name_drop = [col for col in fin_train.columns
                    if re.match(r'^e_week_name_', col)]

# train 해당 컬럼 삭제
fin_train = fin_train.drop(columns=e_weekday_drop)
fin_train = fin_train.drop(columns=e_week_name_drop)

# test c_num로 시작하는 컬럼명 추출
e_weekday_drop = [col for col in fin_test.columns
                  if re.match(r'^e_weekday_', col)]
e_week_name_drop = [col for col in fin_test.columns
                    if re.match(r'^e_week_name_', col)]

# test 해당 컬럼 삭제
fin_test = fin_test.drop(columns=e_weekday_drop)
fin_test = fin_test.drop(columns=e_week_name_drop)

In [ ]:
# train 데이터 (label 포함)
# c_num별 기온관련 모든 열의 평균을 계산하고 새로운 열로 추가
hm_columns = fin_train.filter(regex='^c_nph_hm_').columns
fin_train['mean_c_nph_hm'] = fin_train[hm_columns].mean(axis=1)

ws_10m_columns = fin_train.filter(regex='^c_nph_ws_10m_').columns
fin_train['mean_c_nph_ws_10m'] = fin_train[ws_10m_columns].mean(axis=1)

rn_60m_columns = fin_train.filter(regex='^c_nph_rn_60m_').columns
fin_train['mean_c_nph_rn_60m'] = fin_train[rn_60m_columns].mean(axis=1)

pca_feature_columns = fin_train.filter(regex='^pca_feature_').columns
fin_train['mean_pca_feature'] = fin_train[pca_feature_columns].mean(axis=1)

e_elec_columns = fin_train.filter(regex='^e_elec_').columns
fin_train['mean_e_elec'] = fin_train[e_elec_columns].mean(axis=1)

# 'c_num'별 모든 열을 삭제
fin_train.drop(columns=hm_columns, inplace=True)

fin_train.drop(columns=ws_10m_columns, inplace=True)

fin_train.drop(columns=rn_60m_columns, inplace=True)

fin_train.drop(columns=pca_feature_columns, inplace=True)

fin_train.drop(columns=e_elec_columns, inplace=True)

In [ ]:
# test 데이터 (label 포함)
# c_num별 기온관련 모든 열의 평균을 계산하고 새로운 열로 추가
hm_columns = fin_test.filter(regex='^c_nph_hm_').columns
fin_test['mean_c_nph_hm'] = fin_test[hm_columns].mean(axis=1)

ws_10m_columns = fin_test.filter(regex='^c_nph_ws_10m_').columns
fin_test['mean_c_nph_ws_10m'] = fin_test[ws_10m_columns].mean(axis=1)

rn_60m_columns = fin_test.filter(regex='^c_nph_rn_60m_').columns
fin_test['mean_c_nph_rn_60m'] = fin_test[rn_60m_columns].mean(axis=1)

pca_feature_columns = fin_test.filter(regex='^pca_feature_').columns
fin_test['mean_pca_feature'] = fin_test[pca_feature_columns].mean(axis=1)

# 'c_num'별 모든 열을 삭제
fin_test.drop(columns=hm_columns, inplace=True)

fin_test.drop(columns=ws_10m_columns, inplace=True)

fin_test.drop(columns=rn_60m_columns, inplace=True)

fin_test.drop(columns=pca_feature_columns, inplace=True)

In [ ]:
print(fin_train.shape)
print(fin_test.shape)

(26304, 5)
(8760, 4)


In [ ]:
train_weekday.columns = ['e_weekday']
train_weekname.columns = ['e_weekname']
test_weekday.columns = ['e_weekday']
test_weekname.columns = ['e_weekname']

train_weekday = train_weekday.astype(object)
test_weekday = test_weekday.astype(object)

In [ ]:
train_weekday

,e_weekday
e_tm,
2020-01-01 01:00:00,2
2020-01-01 02:00:00,2
2020-01-01 03:00:00,2
2020-01-01 04:00:00,2
2020-01-01 05:00:00,2
...,...
2022-12-31 20:00:00,5
2022-12-31 21:00:00,5
2022-12-31 22:00:00,5


In [ ]:
# Pandas OneHotEncoding
# 범주형데이터인 weekday(0-6) 원-핫인코딩 진행

train_onehot = pd.get_dummies(train_weekday).astype(int)
test_onehot = pd.get_dummies(test_weekday).astype(int)

In [ ]:
train_onehot.columns

Index(['e_weekday_0', 'e_weekday_1', 'e_weekday_2', 'e_weekday_3',
       'e_weekday_4', 'e_weekday_5', 'e_weekday_6'],
      dtype='object')

In [ ]:
# min max scaling
scaler = MinMaxScaler()

scaled_train_df = scaler.fit_transform(fin_train)
scaled_test_df = scaler.fit_transform(fin_test)

fin_train = pd.DataFrame(scaled_train_df, columns = fin_train.columns)
fin_test = pd.DataFrame(scaled_test_df, columns = fin_test.columns)

## 타겟변수(e_elec) 추출 후 ‘y_train’ 생성

In [ ]:
y_train = fin_train["mean_e_elec"]

del fin_train["mean_e_elec"]

## 최종 데이터프레임 생성

In [ ]:
print(fin_train.shape) #e_tm이 인덱스로 설정 안되어있음
print(train_onehot.shape) #e_tm이 인덱스로 설정
print(train_weekname.shape) #e_tm이 인덱스로 설정

(26304, 4)
(26304, 7)
(26304, 1)


In [ ]:
# 최종 데이터셋으로 concat하기 위해 인덱스 reset
train_onehot = train_onehot.reset_index()
del train_onehot["e_tm"]
print(train_onehot)

train_weekname = train_weekname.reset_index()
del train_weekname["e_tm"]
print(train_weekname)

       e_weekday_0  e_weekday_1  e_weekday_2  e_weekday_3  e_weekday_4  \
0                0            0            1            0            0   
1                0            0            1            0            0   
2                0            0            1            0            0   
3                0            0            1            0            0   
4                0            0            1            0            0   
...            ...          ...          ...          ...          ...   
26299            0            0            0            0            0   
26300            0            0            0            0            0   
26301            0            0            0            0            0   
26302            0            0            0            0            0   
26303            0            0            0            0            0   

       e_weekday_5  e_weekday_6  
0                0            0  
1                0            0  
2        

In [ ]:
test_onehot = test_onehot.reset_index()
del test_onehot["e_tm"]
print(test_onehot)

test_weekname = test_weekname.reset_index()
del test_weekname["e_tm"]
print(test_weekname)

      e_weekday_0  e_weekday_1  e_weekday_2  e_weekday_3  e_weekday_4  \
0               0            0            0            0            0   
1               0            0            0            0            0   
2               0            0            0            0            0   
3               0            0            0            0            0   
4               0            0            0            0            0   
...           ...          ...          ...          ...          ...   
8755            0            0            0            0            0   
8756            0            0            0            0            0   
8757            0            0            0            0            0   
8758            0            0            0            0            0   
8759            1            0            0            0            0   

      e_weekday_5  e_weekday_6  
0               0            1  
1               0            1  
2               0       

In [ ]:
X_train = pd.concat([fin_train, train_onehot, train_weekname], axis=1)
X_test = pd.concat([fin_test, test_onehot, test_weekname], axis = 1)

In [ ]:
print(X_train.shape)
print(X_test.shape)

(26304, 12)
(8760, 12)


In [ ]:
# 'e_tm'만 가지는 데이터프레임 생성
train_time = tmp1[tmp1['c_num'] == 9884]
real_train_time = train_time["e_tm"]
print(real_train_time.head())

test_time = tmp2[tmp2["c_num"] == 9884]
real_test_time = test_time["e_tm"]
print(real_test_time.head())

184128    2020.1.1 1:00
184129    2020.1.1 2:00
184130    2020.1.1 3:00
184131    2020.1.1 4:00
184132    2020.1.1 5:00
Name: e_tm, dtype: object
35040    2023-01-01 1:00
35041    2023-01-01 2:00
35042    2023-01-01 3:00
35043    2023-01-01 4:00
35044    2023-01-01 5:00
Name: e_tm, dtype: object


In [ ]:
#concat위해 인덱스 reset
real_train_time = real_train_time.reset_index()
del real_train_time["index"]
print(real_train_time)

real_test_time = real_test_time.reset_index()
del real_test_time["index"]
print(real_test_time)

                   e_tm
0         2020.1.1 1:00
1         2020.1.1 2:00
2         2020.1.1 3:00
3         2020.1.1 4:00
4         2020.1.1 5:00
...                 ...
26299  2022.12.31 20:00
26300  2022.12.31 21:00
26301  2022.12.31 22:00
26302  2022.12.31 23:00
26303     2023.1.1 0:00

[26304 rows x 1 columns]
                  e_tm
0      2023-01-01 1:00
1      2023-01-01 2:00
2      2023-01-01 3:00
3      2023-01-01 4:00
4      2023-01-01 5:00
...                ...
8755  2023-12-31 20:00
8756  2023-12-31 21:00
8757  2023-12-31 22:00
8758  2023-12-31 23:00
8759   2024-01-01 0:00

[8760 rows x 1 columns]


In [ ]:
print(X_test.head())
print(X_train.head())

   mean_c_nph_hm  mean_c_nph_ws_10m  mean_c_nph_rn_60m  mean_pca_feature  \
0         0.6683             0.2430             0.0000            0.6948   
1         0.6534             0.2613             0.0000            0.6993   
2         0.6449             0.2690             0.0000            0.6931   
3         0.6946             0.2179             0.0000            0.7004   
4         0.7118             0.2536             0.0000            0.7097   

   e_weekday_0  e_weekday_1  e_weekday_2  e_weekday_3  e_weekday_4  \
0            0            0            0            0            0   
1            0            0            0            0            0   
2            0            0            0            0            0   
3            0            0            0            0            0   
4            0            0            0            0            0   

   e_weekday_5  e_weekday_6  e_weekname  
0            0            1           1  
1            0            1           

In [ ]:
X_train = pd.concat([X_train, real_train_time], axis = 1)
y_train = pd.concat([y_train, real_train_time], axis = 1)
X_test = pd.concat([X_test, real_test_time], axis = 1)

In [ ]:
X_train['e_tm'] = pd.to_datetime(X_train['e_tm'])
y_train['e_tm'] = pd.to_datetime(y_train['e_tm'])
X_test['e_tm'] = pd.to_datetime(X_test['e_tm'])

X_train.set_index('e_tm', inplace=True)
y_train.set_index('e_tm', inplace=True)
X_test.set_index('e_tm', inplace=True)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(26304, 12)
(26304, 1)
(8760, 12)


In [ ]:
# Train Data를 학습 및 평가용 Train, Val, Test로 분류
x = X_train
y = y_train

In [ ]:
# train data
x_train = x['2020-01-01':'2021-12-31']
y_train = y['2020-01-01':'2021-12-31']

x_val = x['2022-01-01':'2022-06-30']
y_val = y['2022-01-01':'2022-06-30']

x_test = x['2022-07-01':'2022-12-31']
y_test = y['2022-07-01':'2022-12-31']


In [ ]:
y_train.shape

(26304, 1)

In [ ]:
X_train.head()

,mean_c_nph_hm,mean_c_nph_ws_10m,mean_c_nph_rn_60m,mean_pca_feature,e_weekday_0,e_weekday_1,e_weekday_2,e_weekday_3,e_weekday_4,e_weekday_5,e_weekday_6,e_weekname
e_tm,,,,,,,,,,,,
2020-01-01 01:00:00,0.5531,0.0983,0.0000,0.7829,0,0,1,0,0,0,0,0
2020-01-01 02:00:00,0.5545,0.0749,0.0000,0.7877,0,0,1,0,0,0,0,0
2020-01-01 03:00:00,0.5693,0.0683,0.0000,0.7687,0,0,1,0,0,0,0,0
2020-01-01 04:00:00,0.5944,0.0641,0.0000,0.7689,0,0,1,0,0,0,0,0
2020-01-01 05:00:00,0.6178,0.0441,0.0000,0.7551,0,0,1,0,0,0,0,0


In [ ]:
y_train.head()

,mean_e_elec
e_tm,
2020-01-01 01:00:00,0.2343
2020-01-01 02:00:00,0.1891
2020-01-01 03:00:00,0.1370
2020-01-01 04:00:00,0.1001
2020-01-01 05:00:00,0.0861


In [ ]:
X_test.head()

,mean_c_nph_hm,mean_c_nph_ws_10m,mean_c_nph_rn_60m,mean_pca_feature,e_weekday_0,e_weekday_1,e_weekday_2,e_weekday_3,e_weekday_4,e_weekday_5,e_weekday_6,e_weekname
e_tm,,,,,,,,,,,,
2023-01-01 01:00:00,0.6683,0.2430,0.0000,0.6948,0,0,0,0,0,0,1,1
2023-01-01 02:00:00,0.6534,0.2613,0.0000,0.6993,0,0,0,0,0,0,1,1
2023-01-01 03:00:00,0.6449,0.2690,0.0000,0.6931,0,0,0,0,0,0,1,1
2023-01-01 04:00:00,0.6946,0.2179,0.0000,0.7004,0,0,0,0,0,0,1,1
2023-01-01 05:00:00,0.7118,0.2536,0.0000,0.7097,0,0,0,0,0,0,1,1


## 최종 데이터프레임 csv파일로 저장

In [ ]:
# 전처리 후 생성 데이터셋 csv파일로 저장
X_train.to_csv("X_train.csv")
X_test.to_csv("X_test.csv")
y_train.to_csv("y_train.csv")

<h1>모델1 - RNN</h1>

In [ ]:
import numpy as np

# Convert DataFrame to numpy array
x_train_array = x_train.values
x_val_array = x_val.values
x_test_array = x_test.values
y_test_array = y_test.values

# Reshape the numpy arrays
x_train = x_train_array.reshape(x_train_array.shape[0], 1, x_train_array.shape[1])
x_val = x_val_array.reshape(x_val_array.shape[0], 1, x_val_array.shape[1])
x_test = x_test_array.reshape(x_test_array.shape[0], 1, x_test_array.shape[1])
y_test = y_test_array.reshape(y_test_array.shape[0], 1, y_test_array.shape[1])

In [ ]:

# Define the search space for hyperparameters
space = {
    'units': hp.choice('units', [64, 128, 256]),
    'dropout': hp.uniform('dropout', 0.1, 0.5),
    'recurrent_dropout': hp.uniform('recurrent_dropout', 0.1, 0.3),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.1)),
    'batch_size': hp.choice('batch_size', [16, 32, 64]),
    'epochs': 50,
    'data': (x_train, y_train, x_val, y_val, x_test, y_test)
}

In [ ]:

# Define the objective function to minimize (validation loss)
def objective(params, y_test):
    model = Sequential([
        SimpleRNN(units=params['units'], input_shape=(1, 12), return_sequences=True, dropout=params['dropout'], recurrent_dropout=params['recurrent_dropout']),
        SimpleRNN(units=params['units'] // 2, return_sequences=True, dropout=params['dropout'], recurrent_dropout=params['recurrent_dropout']),
        SimpleRNN(units=params['units'] // 4, return_sequences=False, dropout=params['dropout'], recurrent_dropout=params['recurrent_dropout']),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mean_absolute_error"])
    # Fit the model
    history = model.fit(x_train, y_train,
                        epochs=params['epochs'],
                        batch_size=params['batch_size'],
                        validation_data=(x_val, y_val),
                        verbose=0)

    # y_pred
    y_pred = model.predict(x_test)

    # Flatten y_pred and y_test to 2D
    y_pred = y_pred.reshape(-1)
    y_test = y_test.reshape(-1)

    # Calculate MAE
    mae = mean_absolute_error(y_test, y_pred)

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    # R2
    r2 = r2_score(y_test, y_pred)

    return {'loss': mae, 'status': 'ok', 'rmse': rmse, 'r2': r2}
    return val_loss


In [ ]:
trials = Trials()
best = fmin(fn=lambda params: objective(params, y_test), space=space, algo=tpe.suggest, max_evals=10, trials=trials)
print("Best hyperparameters:", best)

In [ ]:
#optimal hyperparam 'batch_size': 1, 'dropout': 0.25138108140486515, 'learning_rate': 0.027477791611987844, 'recurrent_dropout': 0.264359181620216, 'units': 2}

In [ ]:
from keras.callbacks import EarlyStopping

# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

params = {
    'units': 256,
    'dropout': 0.25138108140486515,
    'recurrent_dropout': 0.264359181620216,
    'learning_rate': 0.027477791611987844 ,
    'batch_size': 32,
    'epochs': 50,
}


model = Sequential([
    SimpleRNN(units=params['units'], input_shape=(1, 12), return_sequences=True, dropout=params['dropout'], recurrent_dropout=params['recurrent_dropout']),
    SimpleRNN(units=params['units'] // 2, return_sequences=True, dropout=params['dropout'], recurrent_dropout=params['recurrent_dropout']),
    SimpleRNN(units=params['units'] // 4, return_sequences=False, dropout=params['dropout'], recurrent_dropout=params['recurrent_dropout']),
    Dense(1, activation='linear')
])

model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mean_absolute_error", "mean_squared_error"])

# Fit the model
history = model.fit(x_train, y_train,
                    epochs=params['epochs'],
                    batch_size=params['batch_size'],
                    validation_data=(x_val, y_val),
                    verbose=0,
                    callbacks=[early_stopping])

# Predict and calculate MAE and RMSE
y_pred = model.predict(x_test)

# Flatten y_pred and y_test to 2D
y_pred = y_pred.reshape(-1)
y_test = y_test.reshape(-1)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

# Save MAE and RMSE
results = {'MAE': mae, 'RMSE': rmse, 'R2': r2}
print(results)


In [ ]:
import matplotlib.pyplot as plt

his_dict = history.history
loss = his_dict['loss']
val_loss = his_dict['val_loss']

epochs = range(1, len(loss) + 1)
fig = plt.figure(figsize = (15,5))

ax1=fig.add_subplot(1,3,1)
ax1.plot(epochs, loss, color ='blue', label='Train')
ax1.plot(epochs, val_loss, color ='orange', label='Test')
ax1.set_title('Loss')
ax1.set_xlabel('epochs')
ax1.set_ylabel('loss')
ax1.legend()

mae = his_dict['mean_absolute_error']
val_mae = his_dict['val_mean_absolute_error']

ax2=fig.add_subplot(1,3,2)
ax2.plot(epochs, mae, color ='blue', label='Train_mae')
ax2.plot(epochs, val_mae, color ='orange', label='Val_mae')
ax2.set_title('Mean Absolute Error')
ax2.set_xlabel('epochs')
ax2.set_ylabel('mean_absolute_error')
ax2.legend()

mse = his_dict['mean_squared_error']
val_mse = his_dict['val_mean_squared_error']

ax3=fig.add_subplot(1,3,3)
ax3.plot(epochs, mse, color ='blue', label='Train_mse')
ax3.plot(epochs, val_mse, color ='orange', label='Val_mse')
ax3.set_title('Mean Squared Error')
ax3.set_xlabel('epochs')
ax3.set_ylabel('mean_squared_error')
ax3.legend()

In [ ]:
pred= model.predict(x_test)

y_pred = y_pred.reshape(-1)  # (n_samples,) 형태로 변환
y_test = y_test.reshape(-1)  # (n_samples,) 형태로 변환

# 3. 예측값과 실제값 비교하는 그래프 그리기
plt.figure(figsize=(20, 6))
plt.plot(y_test, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.title('Comparison of Actual and Predicted Values')
plt.xlabel('Sample')
plt.ylabel('Value')
plt.legend()
plt.show()

<h1>모델2 - LSTM</h1>

In [ ]:
import numpy as np

# Convert DataFrame to numpy array
x_train_array = x_train.values
x_val_array = x_val.values
x_test_array = x_test.values
y_test_array = y_test.values

# Reshape the numpy arrays
x_train = x_train_array.reshape(x_train_array.shape[0], 1, x_train_array.shape[1])
x_val = x_val_array.reshape(x_val_array.shape[0], 1, x_val_array.shape[1])
x_test = x_test_array.reshape(x_test_array.shape[0], 1, x_test_array.shape[1])
y_test = y_test_array.reshape(y_test_array.shape[0], 1, y_test_array.shape[1])

In [ ]:

# Define the search space for hyperparameters
space = {
    'units': hp.choice('units', [64, 128, 256]),
    'dropout': hp.uniform('dropout', 0.1, 0.5),
    'recurrent_dropout': hp.uniform('recurrent_dropout', 0.1, 0.3),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.1)),
    'batch_size': hp.choice('batch_size', [16, 32, 64]),
    'epochs': 100,
    'data': (x_train, y_train, x_val, y_val, x_test, y_test)
}

In [ ]:

# Define the objective function to minimize (validation loss)
def objective(params, y_test):
    model = Sequential([
        LSTM(units=params['units'], input_shape=(1, 12), return_sequences=True, dropout=params['dropout'], recurrent_dropout=params['recurrent_dropout']),
        LSTM(units=params['units'] // 2, return_sequences=True, dropout=params['dropout'], recurrent_dropout=params['recurrent_dropout']),
        LSTM(units=params['units'] // 4, return_sequences=False, dropout=params['dropout'], recurrent_dropout=params['recurrent_dropout']),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mean_absolute_error"])
    # Fit the model
    history = model.fit(x_train, y_train,
                        epochs=params['epochs'],
                        batch_size=params['batch_size'],
                        validation_data=(x_val, y_val),
                        verbose=0)

    # y_pred
    y_pred = model.predict(x_test)

    # Flatten y_pred and y_test to 2D
    y_pred = y_pred.reshape(-1)
    y_test = y_test.reshape(-1)

    # Calculate MAE
    mae = mean_absolute_error(y_test, y_pred)

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    # R2
    r2 = r2_score(y_test, y_pred)

    return {'loss': mae, 'status': 'ok', 'rmse': rmse, 'r2': r2}
    return val_loss


In [ ]:
trials = Trials()
best = fmin(fn=lambda params: objective(params, y_test), space=space, algo=tpe.suggest, max_evals=10, trials=trials)
print("Best hyperparameters:", best)

In [ ]:
# optimal hyperpram #'batch_size': 0, 'dropout': 0.1452954854962082, 'learning_rate': 0.000795870604069595, 'recurrent_dropout': 0.13322767110818748, 'units': 1}

In [ ]:
from keras.callbacks import EarlyStopping

# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

params = {
    'units': 128,
    'dropout': 0.1452954854962082,
    'recurrent_dropout': 0.13322767110818748,
    'learning_rate': 0.000795870604069595 ,
    'batch_size': 16,
    'epochs': 200,
}


model = Sequential([
    LSTM(units=params['units'], input_shape=(1, 12), return_sequences=True, dropout=params['dropout'], recurrent_dropout=params['recurrent_dropout']),
    LSTM(units=params['units'] // 2, return_sequences=True, dropout=params['dropout'], recurrent_dropout=params['recurrent_dropout']),
    LSTM(units=params['units'] // 4, return_sequences=False, dropout=params['dropout'], recurrent_dropout=params['recurrent_dropout']),
    Dense(1, activation='linear')
])

model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mean_absolute_error", "mean_squared_error"])

# Fit the model
history = model.fit(x_train, y_train,
                    epochs=params['epochs'],
                    batch_size=params['batch_size'],
                    validation_data=(x_val, y_val),
                    verbose=0,
                    callbacks = [early_stopping])

# Predict and calculate MAE and RMSE
y_pred = model.predict(x_test)

# Flatten y_pred and y_test to 2D
y_pred = y_pred.reshape(-1)
y_test = y_test.reshape(-1)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

# Save MAE and RMSE
results = {'MAE': mae, 'RMSE': rmse, 'R2': r2}
print(results)


In [ ]:
import matplotlib.pyplot as plt

his_dict = history.history
loss = his_dict['loss']
val_loss = his_dict['val_loss']

epochs = range(1, len(loss) + 1)
fig = plt.figure(figsize = (15,5))

ax1=fig.add_subplot(1,3,1)
ax1.plot(epochs, loss, color ='blue', label='Train')
ax1.plot(epochs, val_loss, color ='orange', label='Test')
ax1.set_title('Loss')
ax1.set_xlabel('epochs')
ax1.set_ylabel('loss')
ax1.legend()

mae = his_dict['mean_absolute_error']
val_mae = his_dict['val_mean_absolute_error']

ax2=fig.add_subplot(1,3,2)
ax2.plot(epochs, mae, color ='blue', label='Train_mae')
ax2.plot(epochs, val_mae, color ='orange', label='Val_mae')
ax2.set_title('Mean Absolute Error')
ax2.set_xlabel('epochs')
ax2.set_ylabel('mean_absolute_error')
ax2.legend()

mse = his_dict['mean_squared_error']
val_mse = his_dict['val_mean_squared_error']

ax3=fig.add_subplot(1,3,3)
ax3.plot(epochs, mse, color ='blue', label='Train_mse')
ax3.plot(epochs, val_mse, color ='orange', label='Val_mse')
ax3.set_title('Mean Squared Error')
ax3.set_xlabel('epochs')
ax3.set_ylabel('mean_squared_error')
ax3.legend()

In [ ]:
# 예측값과 실제값 산포도 시각화 (R-squared 포함)
plt.figure(figsize=(8, 8))
plt.scatter(y_test, y_pred, alpha=0.3, label='Data points')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label='Ideal fit')
plt.title(f'True vs Predicted Scatter Plot\nR-squared: {r2:.3f}')
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.legend()
plt.show()

<h1>모델3 - GRU</h1>

In [ ]:

# Define the search space for hyperparameters
space = {
    'units': hp.choice('units', [64, 128, 256]),
    'dropout': hp.uniform('dropout', 0.1, 0.5),
    'recurrent_dropout': hp.uniform('recurrent_dropout', 0.1, 0.3),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.1)),
    'batch_size': hp.choice('batch_size', [16, 32, 64]),
    'epochs': 50,
    'data': (x_train, y_train, x_val, y_val, x_test, y_test)
}

In [ ]:

# Define the objective function to minimize (validation loss)
def objective(params, y_test):
    model = Sequential([
        SimpleRNN(units=params['units'], input_shape=(1, 12), return_sequences=True, dropout=params['dropout'], recurrent_dropout=params['recurrent_dropout']),
        SimpleRNN(units=params['units'] // 2, return_sequences=True, dropout=params['dropout'], recurrent_dropout=params['recurrent_dropout']),
        SimpleRNN(units=params['units'] // 4, return_sequences=False, dropout=params['dropout'], recurrent_dropout=params['recurrent_dropout']),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mean_absolute_error"])
    # Fit the model
    history = model.fit(x_train, y_train,
                        epochs=params['epochs'],
                        batch_size=params['batch_size'],
                        validation_data=(x_val, y_val),
                        verbose=0)

    # y_pred
    y_pred = model.predict(x_test)

    # Flatten y_pred and y_test to 2D
    y_pred = y_pred.reshape(-1)
    y_test = y_test.reshape(-1)

    # Calculate MAE
    mae = mean_absolute_error(y_test, y_pred)

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    # R2
    r2 = r2_score(y_test, y_pred)

    return {'loss': mae, 'status': 'ok', 'rmse': rmse, 'r2': r2}
    return val_loss


In [ ]:
trials = Trials()
best = fmin(fn=lambda params: objective(params, y_test), space=space, algo=tpe.suggest, max_evals=10, trials=trials)
print("Best hyperparameters:", best)

In [ ]:
#optimal hyperparam 'batch_size': 1, 'dropout': 0.25138108140486515, 'learning_rate': 0.027477791611987844, 'recurrent_dropout': 0.264359181620216, 'units': 2}

In [ ]:
from keras.callbacks import EarlyStopping

# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

params = {
    'units': 256,
    'dropout': 0.25138108140486515,
    'recurrent_dropout': 0.264359181620216,
    'learning_rate': 0.027477791611987844 ,
    'batch_size': 32,
    'epochs': 200,
}


model = Sequential([
    SimpleRNN(units=params['units'], input_shape=(1, 12), return_sequences=True, dropout=params['dropout'], recurrent_dropout=params['recurrent_dropout']),
    SimpleRNN(units=params['units'] // 2, return_sequences=True, dropout=params['dropout'], recurrent_dropout=params['recurrent_dropout']),
    SimpleRNN(units=params['units'] // 4, return_sequences=False, dropout=params['dropout'], recurrent_dropout=params['recurrent_dropout']),
    Dense(1, activation='linear')
])

model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mean_absolute_error", "mean_squared_error"])

# Fit the model
history = model.fit(x_train, y_train,
                    epochs=params['epochs'],
                    batch_size=params['batch_size'],
                    validation_data=(x_val, y_val),
                    verbose=0,
                    callbacks=[early_stopping])

# Predict and calculate MAE and RMSE
y_pred = model.predict(x_test)

# Flatten y_pred and y_test to 2D
y_pred = y_pred.reshape(-1)
y_test = y_test.reshape(-1)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

# Save MAE and RMSE
results = {'MAE': mae, 'RMSE': rmse, 'R2': r2}
print(results)


In [ ]:
import matplotlib.pyplot as plt

his_dict = history.history
loss = his_dict['loss']
val_loss = his_dict['val_loss']

epochs = range(1, len(loss) + 1)
fig = plt.figure(figsize = (15,5))

ax1=fig.add_subplot(1,3,1)
ax1.plot(epochs, loss, color ='blue', label='Train')
ax1.plot(epochs, val_loss, color ='orange', label='Test')
ax1.set_title('Loss')
ax1.set_xlabel('epochs')
ax1.set_ylabel('loss')
ax1.legend()

mae = his_dict['mean_absolute_error']
val_mae = his_dict['val_mean_absolute_error']

ax2=fig.add_subplot(1,3,2)
ax2.plot(epochs, mae, color ='blue', label='Train_mae')
ax2.plot(epochs, val_mae, color ='orange', label='Val_mae')
ax2.set_title('Mean Absolute Error')
ax2.set_xlabel('epochs')
ax2.set_ylabel('mean_absolute_error')
ax2.legend()

mse = his_dict['mean_squared_error']
val_mse = his_dict['val_mean_squared_error']

ax3=fig.add_subplot(1,3,3)
ax3.plot(epochs, mse, color ='blue', label='Train_mse')
ax3.plot(epochs, val_mse, color ='orange', label='Val_mse')
ax3.set_title('Mean Squared Error')
ax3.set_xlabel('epochs')
ax3.set_ylabel('mean_squared_error')
ax3.legend()

In [ ]:
# 예측값과 실제값 산포도 시각화 (R-squared 포함)
plt.figure(figsize=(8, 8))
plt.scatter(y_test, y_pred, alpha=0.3, label='Data points')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label='Ideal fit')
plt.title(f'True vs Predicted Scatter Plot\nR-squared: {r2:.3f}')
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.legend()
plt.show()

In [ ]:
pred= model.predict(x_test)

y_pred = y_pred.reshape(-1)  # (n_samples,) 형태로 변환
y_test = y_test.reshape(-1)  # (n_samples,) 형태로 변환

# 3. 예측값과 실제값 비교하는 그래프 그리기
plt.figure(figsize=(20, 6))
plt.plot(y_test, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.title('Comparison of Actual and Predicted Values')
plt.xlabel('Sample')
plt.ylabel('Value')
plt.legend()
plt.show()